In [1]:
from opentrons import robot, containers, instruments
import pandas as pd
import argparse
import sys

# Configuration
SOURCE_SLOTS = ['D1','D2','D3','B2']

## Load our files
#parser = argparse.ArgumentParser(description="Run a DNA build on an Opentrons OT-1 robot.")
#parser.add_argument('-l', '--layout', required=False, help="A CSV file describing the layout of the sourcep plates.")
#parser.add_argument('-b', '--build-plan', required=True, help="A CSV file describing the build plan.")
#parser.add_argument('-s', '--simulate', required=False, action="store_true", help="Simulate the robot run and print command output.")
#args = parser.parse_args()

#plan = pd.read_csv(args.build_plan, usecols=['Gene','Wells'])
plan = pd.read_csv('../synth1/buildable-fragments_39-50.csv', usecols=['Gene','Wells'])



if 1 == 0: #args.layout:
    # We were given an explicit layout
    layout = pd.read_csv(args.layout)
    layout = layout.set_index('Position').to_dict()['Name'] # Turn into a location->name dict
else:
    plate_names = plan['Wells'].str.split('-').str[0].unique()

    if len(plate_names) > len(SOURCE_SLOTS):
        print("Error: This build plan requires too many source plates.")
        sys.exit(1)

    layout = list(zip(SOURCE_SLOTS[:len(plate_names)], plate_names))

    slots = pd.Series(SOURCE_SLOTS)
    columns = sorted(slots.str[0].unique())
    rows = sorted(slots.str[1].unique(), reverse=True)

    layout_table = pd.DataFrame(index=rows, columns=columns)
    layout_table.fillna("", inplace=True)

    for slot, plate in layout:
        layout_table.loc[slot[1], slot[0]] = plate

    print("Please arrange the plates in the following configuration:")
    print()
    print(layout_table)
    print()
    input("Press enter to continue")

    layout = dict(layout)


Please arrange the plates in the following configuration:

                    B                   D
3                      pSHPs0826B426849MU
2  pSHPs0826B426850MU  pSHPs0807B412040MU
1                      pSHPs0807B412039MU

Press enter to continue


In [2]:
# Configure the robot

#  Layout:
#    A     B       C      D      E
#  3 p200  master  master source p10
#  2       dest    dest   source p10
#  1       trash          source p10
#

if 1 == 1: #args.simulate:
    print("Simulating protcol run")
    robot.connect()
else:
    port = robot.get_serial_ports_list()[0]
    print("Connecting robot to port {}".format(port))
    robot.connect(port)

robot.home()

p200_tipracks = [
    containers.load('tiprack-200ul', 'A3'),
]

p10_tipracks = [
    containers.load('tiprack-10ul', 'E2'),
]

p10s_tipracks = [
    containers.load('tiprack-10ul', 'E3'),
    containers.load('tiprack-10ul', 'E1'),
]

trash = containers.load('point', 'B1', 'holywastedplasticbatman')
master = containers.load('PCR-strip-tall', 'C3')

dest_plates = [
    containers.load('96-PCR-tall', 'C2'),
    containers.load('96-PCR-tall', 'B2')
]

source_plates = {}
for slot, plate in layout.items():
    source_plates[plate] = containers.load('96-flat', slot)

p10 = instruments.Pipette(
    axis='a',
    max_volume=10,
    min_volume=0.5,
    tip_racks=p10_tipracks,
    trash_container=trash,
    channels=8,
    name='p10-8'
)

p10s = instruments.Pipette(
    axis='a',
    max_volume=10,
    min_volume=0.5,
    tip_racks=p10s_tipracks,
    trash_container=trash,
    channels=1,
    name='p10-8s'
)

p200 = instruments.Pipette(
    axis='b',
    max_volume=200,
    min_volume=20,
    tip_racks=p200_tipracks,
    trash_container=trash,
    channels=1,
    name='p200-1'
)


Simulating protcol run


In [3]:
# Run the protocol

# Load dest plates

# Distribute down the plates

max_reactions = 96

if len(plan) <= max_reactions:
    num_reactions = len(plan)
    print("Will run {} reactions".format(num_reactions))
else:
    num_reactions = max_reactions
    print("Too many reactions, will only run first {} reactions".format(num_reactions))

#num_reactions = len(plan)
num_rows = num_reactions // 8
print(num_rows)
all_wells = dest_plates[0].wells() + dest_plates[1].wells()

print("Building {} reactions in {} rows".format(num_reactions, num_rows))

p10.pick_up_tip()

for i in range(num_rows):
    p = i // 12
    r = i % 12

    print("Transferring master mix to plate {} row {}".format(p, r))
    p10.transfer(8, master['A1'], dest_plates[p].rows(r).bottom(), blow_out=True, touch_tip=True, new_tip='never')

p10.drop_tip()

j = 0

# Add multiples of mastermix to plates with multiple fragments
p10s.pick_up_tip()
for i, construct in plan.iterrows():
    vol = 8 * (len(construct['Wells'].split(',')) - 1)
    j = j + (len(construct['Wells'].split(',')) - 1)
    if vol > 0:
        print("Adding {} ul to well {} for multifragment assembly".format(vol, i))
        p10s.transfer(vol, master['A1'], all_wells[int(i)].bottom(), blow_out=True, touch_tip=True, new_tip='never')
    if i == num_reactions:
        break
p10s.drop_tip()

master_reactions = (num_reactions + j) + 8
print("Make {} rxns of master mix".format(master_reactions))

master_mix = pd.DataFrame({
    'Component':['Cutsmart','ATP','Vector','T4 Ligase','BbsI','H2O','Total'],
    'Amount':[master_reactions,master_reactions,(0.25*master_reactions),(master_reactions*(50/96)),(master_reactions*(6/96)),(5.166*master_reactions),(master_reactions*8)]
})
master_mix
## Move source DNA into dest mastermixes
#for i,construct in plan.iterrows():
#    print("Building gene {} {}".format(i, construct['Gene']))
#    fragments = construct['Wells'].split(',')
#    for fragment in fragments:
#        plate, well = fragment.split('-')
#        print("    Adding fragment from plate {} well {}".format(plate,well))
#        p10s.transfer(2, source_plates[plate].wells(well).bottom(), all_wells[int(i)].bottom(), blow_out=True, touch_tip=True, mix_before=(3,5))
#    if i == num_reactions:
#        break
#if args.simulate:
#    print()
#    print("Ran commands:")
#    for c in robot.commands():
#        print(c)
#

Too many reactions, will only run first 96 reactions
12
Building 96 reactions in 12 rows
Transferring master mix to plate 0 row 0
Transferring master mix to plate 0 row 1
Transferring master mix to plate 0 row 2
Transferring master mix to plate 0 row 3
Transferring master mix to plate 0 row 4
Transferring master mix to plate 0 row 5
Transferring master mix to plate 0 row 6
Transferring master mix to plate 0 row 7
Transferring master mix to plate 0 row 8
Transferring master mix to plate 0 row 9
Transferring master mix to plate 0 row 10
Transferring master mix to plate 0 row 11
Adding 8 ul to well 15 for multifragment assembly
Adding 8 ul to well 18 for multifragment assembly
Adding 8 ul to well 19 for multifragment assembly
Adding 8 ul to well 20 for multifragment assembly
Adding 8 ul to well 35 for multifragment assembly
Adding 8 ul to well 43 for multifragment assembly
Adding 8 ul to well 48 for multifragment assembly
Adding 8 ul to well 58 for multifragment assembly
Adding 8 ul to we

,Amount,Component
0,116.000000,Cutsmart
1,116.000000,ATP
2,29.000000,Vector
3,60.416667,T4 Ligase
4,7.250000,BbsI
5,599.256000,H2O
6,928.000000,Total


In [45]:
plan[:max_reactions].to_csv('../synth1/round1-assemblies.csv')

In [278]:
## FOR GIT REPOSITORY -- Queries the database for build ready constructs and then sets up the plates

from opentrons import robot, containers, instruments
import argparse
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

## Take in required information

# Load files
#parser = argparse.ArgumentParser(description="Resuspend a plate of DNA on an Opentrons OT-1 robot.")
#parser.add_argument('-r', '--run', required=False, action="store_true", help="Send commands to the robot and print command output.")
#args = parser.parse_args()

letter = ["A","B","C","D","E","F","G","H"]
number = ["1","2","3","4","5","6","7","8","9","10","11","12"]

target_well = []
temp_well = 0

for n in number:
    for l in letter:
        temp_well = l + n
        target_well.append(temp_well)

targets = []

counter = 0
frag_num = 0
frag_list = []
master_well = []

unique_plates = 0
number_plates = 3
#plates = ["pSHPs0807B412037MU", "pSHPs0807B412038MU"]
plates = []

gene_list = []
gene_num = 1

# Query the database and iterate through each json file
for file in glob.glob("../data/BBF10K*/*.json"):
    print(file)
    
    # Open and store the data within the json file
    with open(file,"r") as json_file:
        data = json.load(json_file)
    
    # Determine if it is build ready
    if data["status"]["build_ready"] != "TRUE":
        continue
    print("build ready")

    #Set a limit for how many you want to run
    if gene_num == 20:
        break
    
    # Pull general information about the gene
    gene = data["id"]
    locations = data["location"]["fragments"]
    frag_num = len(locations)

    # Set the limit on how many fragments you want to build in one reaction
    if frag_num > 2:
        continue
    print("number of fragments: ", frag_num)
    
    # Start a counter for the fragment number
    frag_count = 0
    
    # Iterate through all of the fragments
    for fragment in locations:    
        print(fragment)
        
        # Sets a skip variable to allow you to skip in a nested for loop
        skip = 0
        
        # Starts the counter at 0 and then increments up
        frag_count = frag_count + 1
        
        # Pulls out the location for a specific fragment
        frag_loc = data["location"]["fragments"][fragment]
        
        # Separates the well from the plate and determines how many unique plates there are
        plate_loc, well = frag_loc.split("_")
        plates.append(plate_loc)
        plates_pd = pd.Series(plates)
        unique_plates = len(plates_pd.unique())
        print("number on unique plates:", unique_plates)
        
        # Determines if there are now too many unique plates
        if unique_plates > number_plates:
            
            # Removes the plate that was added
            plates = plates[:-(frag_count)]
            
            # If a later fragment is not within the plates specified it goes back and removes the earlier genes that were added
            if frag_count != 1:
                gene_list = gene_list[:-(frag_count - 1)]
            print("too many unique plates")
            print()
            
            # Breaks out of the fragment for loop and sets it to skip to the next gene
            skip = 1
            break
        else:
            # If the gene passes all of the checks it is added to the list of genes to be made
            gene_list.append(gene)
            
            # Turn the list into a series and determine the number of unique genes
            gene_list_pd = pd.Series(gene_list)
            gene_num = len(gene_list_pd.unique())
            
            # Attaches a destination well to the target gene
            dest_well = target_well[gene_num - 1]
            
            # Builds a row containing all of the required information for aliquoting the fragments
            row = [gene, plate_loc, well, dest_well]
            targets.append(row)
            print("gene_num", gene_num)
        print()
    # Skips to the next gene if one of the fragments requires a plate that we are not including
    if skip == 1:
        continue
    # If all of the fragments are cleared the number of reactions needed for the master mix is added along with the well it corresponds to
    else:
        frag_list.append(frag_num)
        print("num frag list: ", len(frag_list))
        master_well.append(target_well[gene_num - 1])
        print("num wells: ", len(master_well))
    print()
    print()
print("unique plates: ", plates_pd.unique())
       
# Creates a dataframe with the well and 
master_plan = pd.DataFrame({
    "Well" : master_well,
    "Fragments" : frag_list
})
master_plan = master_plan[["Well","Fragments"]]

targets = np.array(targets)
plan = pd.DataFrame({
    "Plate" : targets[:,1],
    "Gene" : targets[:,0],
    "Well" : targets[:,2],
    "Destination" : targets[:,3]
    })
plan = plan[["Gene","Plate","Well","Destination"]]
print(plan)

print(len(target_well))

master_plan
        


../data/BBF10K_000001/BBF10K_000001.json
build ready
number of fragments:  1
BBF10K_000001_1
number on unique plates: 1
gene_num 1

num frag list:  1
num wells:  1


../data/BBF10K_000002/BBF10K_000002.json
build ready
number of fragments:  1
BBF10K_000002_1
number on unique plates: 2
gene_num 2

num frag list:  2
num wells:  2


../data/BBF10K_000003/BBF10K_000003.json
build ready
number of fragments:  1
BBF10K_000003_1
number on unique plates: 3
gene_num 3

num frag list:  3
num wells:  3


../data/BBF10K_000004/BBF10K_000004.json
build ready
number of fragments:  1
BBF10K_000004_1
number on unique plates: 3
gene_num 4

num frag list:  4
num wells:  4


../data/BBF10K_000005/BBF10K_000005.json
build ready
number of fragments:  1
BBF10K_000005_1
number on unique plates: 4
too many unique plates

../data/BBF10K_000006/BBF10K_000006.json
build ready
number of fragments:  2
BBF10K_000006_1
number on unique plates: 3
gene_num 5

BBF10K_000006_2
number on unique plates: 3
gene_num 5

num f

,Well,Fragments
0,A1,1
1,B1,1
2,C1,1
3,D1,1
4,E1,2
5,F1,2
6,G1,2
7,H1,1
8,A2,1
9,B2,1


In [101]:
array = []
plate = "first"
array.append(plate)
print(array)
array.append("another")
array.append("and")
array.append("last")

print(array[:-1])


['first']
['first', 'another', 'and']


In [281]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

import shutil

import datetime

counter = 0

# Imports the csv containing all of our data
data = pd.read_csv("./testing/data_testing/10K_CDS.csv")

# Takes in the 
for file in glob.glob("./testing/json/template.json"):
        print(file)
        with open(file,"r") as template_json:
            template = json.load(template_json)
            #print(template)

# Iterates through all of the items in the spreadsheet
for index, row in data.iterrows():
    
    # Forces it to only run a certain number of times
    #counter = counter + 1
    #if counter == 11:
    #    break

    # Generate the ID#
    gene = row['gene_name']
    seq = row['sequence']
    number = str(index + 1).zfill(6)
    idnum = "BBF10K_" + number
    
    #State the path to house the new set of directories
    path = "../data/{}".format(idnum)
    
    # Only create a new directory if there are no existing directories
    if os.path.exists(path):
        print("Directory {} already exists".format(idnum))
    else:
        # Generates a new directory with the ID# as its name
        os.makedirs(path)
        
        # Generate the fasta file with the gene sequence
        fasta = open("./{}/{}.fasta".format(path,idnum),"w+")
        fasta.write(">{}".format(gene))
        fasta.write("\n")
        fasta.write(seq)
        fasta.close()
        
        # Fill in the template json file with the information from each row
        template["gene_id"] = idnum
        print(idnum)
        template["gene_name"] = gene
        template["sequence"]["original_sequence"] = seq
        template["sequence"]["optimized_sequence"] = seq
        template["author"]["name"] = row["author"]
        template["author"]["email"] = row["author_email"]
        template["author"]["affiliation"] = row["author_affiliation"]
        template["author"]["project"] = row["author_project"]
        template["info"]["type"]["cloning_method"] = row["cloning_method"]
        template["info"]["type"]["part_type"] = row["part_type"]
        template["info"]["type"]["build_type"] = row["build_type"]
        template["info"]["safety"] = row["safety"]
        template["info"]["collection"] = row["collection"]
        template["info"]["other_tags"] = row["other_tags"]
        template["status"]["ordered"] = row["ordered"]
        template["status"]["will_build"] = row["will_build"]
        date = row["date_ordered"]
        year, month, day = date.split(".")
        week = datetime.date(year, month, day).isocalendar()[1]
        order_week = year + "." + week
        template["info"]["order_number"] = row["order_number"]
        template["info"]["order_week"] = order_week
        template["dates"]["submitted"] = row["date_ordered"]
        template["dates"]["ordered"] = row["date_ordered"]
        
        with open("./{}/{}.json".format(path,idnum),"w+") as json_file:
            json.dump(template,json_file,indent=2)
    
    
    

./testing/json/template.json


In [215]:
letter = ["A","B","C","D","E","F","G","H"]
number = ["1","2","3","4","5","6","7","8","9","10","11","12"]

target_well = []
temp_well = 0

for n in number:
    for l in letter:
        temp_well = l + n
        target_well.append(temp_well)
        


D1


In [265]:
## Test build.py - Not functional
from opentrons import robot, containers, instruments
import argparse
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

## Take in required information

# Load files
#parser = argparse.ArgumentParser(description="Resuspend a plate of DNA on an Opentrons OT-1 robot.")
#parser.add_argument('-r', '--run', required=False, action="store_true", help="Send commands to the robot and print command output.")
#args = parser.parse_args()

letter = ["A","B","C","D","E","F","G","H"]
number = ["1","2","3","4","5","6","7","8","9","10","11","12"]

target_well = []
temp_well = 0

for n in number:
    for l in letter:
        temp_well = l + n
        target_well.append(temp_well)
        
database = []        

counter = 0

# Query the database and iterate through each json file
for file in glob.glob("../data/BBF10K*/*.json"):
    print(file)
    counter = counter + 1
    if counter == 100:
        break
    # Open and store the data within the json file
    with open(file,"r") as json_file:
        data = json.load(json_file)
    
    if data["status"]["build_ready"] != "TRUE":
        continue
    print("build ready")
    
    gene = data["id"]
    locations = data["location"]["fragments"]
    frag_num = len(locations)
    print("frag_num", frag_num)

    for fragment in locations:
        # Pull general information about the gene
        frag_loc = data["location"]["fragments"][fragment]
        print(frag_loc)
        plate_loc, well = frag_loc.split("_")
        row = [gene, plate_loc, well, frag_num]
        database.append(row)

database = np.array(database)
database = pd.DataFrame({
    "Gene" : database[:,0],
    "Plate" : database[:,1],
    "Well" : database[:,2],
    "Fragments" : database[:,3]
})
database = database.set_index("Gene")
database

../data/BBF10K_000001/BBF10K_000001.json
build ready
frag_num 1
pSHPs0826B426849MU_B6
../data/BBF10K_000002/BBF10K_000002.json
build ready
frag_num 1
pSHPs0807B412037MU_A1
../data/BBF10K_000003/BBF10K_000003.json
build ready
frag_num 1
pSHPs0807B412038MU_F3
../data/BBF10K_000004/BBF10K_000004.json
build ready
frag_num 1
pSHPs0807B412037MU_A3
../data/BBF10K_000005/BBF10K_000005.json
build ready
frag_num 1
pSHPs0807B412040MU_D2
../data/BBF10K_000006/BBF10K_000006.json
build ready
frag_num 2
pSHPs0807B412037MU_F8
pSHPs0807B412037MU_G8
../data/BBF10K_000007/BBF10K_000007.json
build ready
frag_num 2
pSHPs0807B412037MU_E6
pSHPs0807B412037MU_F6
../data/BBF10K_000008/BBF10K_000008.json
build ready
frag_num 1
pSHPs0807B412039MU_D1
../data/BBF10K_000009/BBF10K_000009.json
build ready
frag_num 2
pSHPs0807B412038MU_D5
pSHPs0826B426849MU_B5
../data/BBF10K_000010/BBF10K_000010.json
../data/BBF10K_000011/BBF10K_000011.json
build ready
frag_num 1
pSHPs0826B426849MU_G8
../data/BBF10K_000012/BBF10K_0000

../data/BBF10K_000100/BBF10K_000100.json


,Fragments,Plate,Well
Gene,,,
BBF10K_000001,1,pSHPs0826B426849MU,B6
BBF10K_000002,1,pSHPs0807B412037MU,A1
BBF10K_000003,1,pSHPs0807B412038MU,F3
BBF10K_000004,1,pSHPs0807B412037MU,A3
BBF10K_000005,1,pSHPs0807B412040MU,D2
BBF10K_000006,2,pSHPs0807B412037MU,F8
BBF10K_000006,2,pSHPs0807B412037MU,G8
BBF10K_000007,2,pSHPs0807B412037MU,E6
BBF10K_000007,2,pSHPs0807B412037MU,F6


In [307]:
## FOR GIT REPOSITORY: Takes in a file containing all of the fragment sequences and then appends the information to the json files

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

import shutil

counter = 0

no_frag = []
linked_genes = []
not_in_dict = []

# Import the Gene -> ID dictionary to allow the use of the gene name
for file in glob.glob("./gene_id_dict.json"):
        print("Opening dictionary file: ", file)
        with open(file,"r") as json_file_dict:
            #print(json_file_dict)
            dictionary = json.load(json_file_dict)

# Imports the csv containing all of our data
data = pd.read_csv("./testing/data_testing/fragments1-5.csv")

# Iterates through all of the items in the spreadsheet
for index, row in data.iterrows():
    
    # Stops it from going through every file in the database
    #counter = counter + 1
    #if counter == 10:
    #    break
    
    # Takes in the name of the fragment and decides if it has a fragment or not
    name = row["Fragment"]
    seq = row["Sequence"]
    if name[-2] != "_":
        no_frag.append(name)
        continue
    
    # Splits the fragment name into gene name and fragment number
    gene = name[:-2]
    fragment = name[-1]
    
    # Checks if it is a linked gene, right now it only takes the first gene
    if 'link' in gene:
        print('link')
        linked_genes.append(gene)
        gene =  gene[:11]    
    
    # Checks if the gene is in the dictionary and if not record it and move on
    if gene not in dictionary:
        print("{} not in dictionary".format(gene))
        not_in_dict.append(gene)
        continue
    
    # Set the ID# based on the dictionary specification
    print("name", name)
    idnum = dictionary[gene]
    id_frag = idnum + "_" + fragment
    print("id_frag", id_frag)
    
    # Look up the specific json file associated with the current fragment
    for file in glob.glob("../data/{}/{}.json".format(idnum,idnum)):
        print(file)
        # Open the json file
        with open(file,"r") as json_file:
            data = json.load(json_file)
        data["location"]["fragments"][id_frag] = ""
        data["sequence"]["fragment_sequences"][id_frag] = seq
        #print(data["location"]["fragments"][name])
        with open(file,'w') as json_file:
                json.dump(data,json_file,indent=2)        
        

print(no_frag)
print(not_in_dict)
    
    
    

Opening dictionary file:  ./gene_id_dict.json
name Alkaline Phosphatase_1
id_frag BBF10K_000439_1
../data/BBF10K_000439/BBF10K_000439.json
name Bst Polymerase(KG)_1
id_frag BBF10K_000440_1
../data/BBF10K_000440/BBF10K_000440.json
name CdnDI_1
id_frag BBF10K_000441_1
../data/BBF10K_000441/BBF10K_000441.json
name Deep_Vent_1
id_frag BBF10K_000450_1
../data/BBF10K_000450/BBF10K_000450.json
name Deep_Vent_2
id_frag BBF10K_000450_2
../data/BBF10K_000450/BBF10K_000450.json
name Deep_Vent_3
id_frag BBF10K_000450_3
../data/BBF10K_000450/BBF10K_000450.json
name lambda exonuclease_1
id_frag BBF10K_000442_1
../data/BBF10K_000442/BBF10K_000442.json
name exonuclease i_1
id_frag BBF10K_000443_1
../data/BBF10K_000443/BBF10K_000443.json
name Pfu_Poly_1
id_frag BBF10K_000451_1
../data/BBF10K_000451/BBF10K_000451.json
name Pfu_Poly_2
id_frag BBF10K_000451_2
../data/BBF10K_000451/BBF10K_000451.json
name T7 ligase_1
id_frag BBF10K_000444_1
../data/BBF10K_000444/BBF10K_000444.json
name T5 exonuclease_1
id_

name gene_8853_2
id_frag BBF10K_000471_2
../data/BBF10K_000471/BBF10K_000471.json
name gene_12442_1
id_frag BBF10K_000472_1
../data/BBF10K_000472/BBF10K_000472.json
name gene_12442_2
id_frag BBF10K_000472_2
../data/BBF10K_000472/BBF10K_000472.json
name gene_12924_1
id_frag BBF10K_000473_1
../data/BBF10K_000473/BBF10K_000473.json
name gene_12924_2
id_frag BBF10K_000473_2
../data/BBF10K_000473/BBF10K_000473.json
name gene_13895_1
id_frag BBF10K_000474_1
../data/BBF10K_000474/BBF10K_000474.json
name gene_13895_2
id_frag BBF10K_000474_2
../data/BBF10K_000474/BBF10K_000474.json
name gene_14034_1
id_frag BBF10K_000475_1
../data/BBF10K_000475/BBF10K_000475.json
name gene_14034_2
id_frag BBF10K_000475_2
../data/BBF10K_000475/BBF10K_000475.json
name gene_14606_1
id_frag BBF10K_000476_1
../data/BBF10K_000476/BBF10K_000476.json
name gene_14606_2
id_frag BBF10K_000476_2
../data/BBF10K_000476/BBF10K_000476.json
name gene_20356_1
id_frag BBF10K_000477_1
../data/BBF10K_000477/BBF10K_000477.json
name 

name gene_16842_2
id_frag BBF10K_000523_2
../data/BBF10K_000523/BBF10K_000523.json
name gene_18839_1
id_frag BBF10K_000524_1
../data/BBF10K_000524/BBF10K_000524.json
name gene_18839_2
id_frag BBF10K_000524_2
../data/BBF10K_000524/BBF10K_000524.json
name gene_18998_1
id_frag BBF10K_000525_1
../data/BBF10K_000525/BBF10K_000525.json
name gene_18998_2
id_frag BBF10K_000525_2
../data/BBF10K_000525/BBF10K_000525.json
name gene_19803_1
id_frag BBF10K_000526_1
../data/BBF10K_000526/BBF10K_000526.json
name gene_19803_2
id_frag BBF10K_000526_2
../data/BBF10K_000526/BBF10K_000526.json
name gene_20968_1
id_frag BBF10K_000527_1
../data/BBF10K_000527/BBF10K_000527.json
name gene_20968_2
id_frag BBF10K_000527_2
../data/BBF10K_000527/BBF10K_000527.json
name gene_2579_1
id_frag BBF10K_000528_1
../data/BBF10K_000528/BBF10K_000528.json
name gene_2579_2
id_frag BBF10K_000528_2
../data/BBF10K_000528/BBF10K_000528.json
name gene_4481_1
id_frag BBF10K_000529_1
../data/BBF10K_000529/BBF10K_000529.json
name ge

name MMSYN1_0365_1
id_frag BBF10K_000160_1
../data/BBF10K_000160/BBF10K_000160.json
name MMSYN1_0117_1
id_frag BBF10K_000049_1
../data/BBF10K_000049/BBF10K_000049.json
name MMSYN1_0613_1
id_frag BBF10K_000272_1
../data/BBF10K_000272/BBF10K_000272.json
name MMSYN1_0666_1
id_frag BBF10K_000306_1
../data/BBF10K_000306/BBF10K_000306.json
name MMSYN1_0795_1
id_frag BBF10K_000349_1
../data/BBF10K_000349/BBF10K_000349.json
name MMSYN1_0198_1
id_frag BBF10K_000082_1
../data/BBF10K_000082/BBF10K_000082.json
name MMSYN1_0076_1
id_frag BBF10K_000034_1
../data/BBF10K_000034/BBF10K_000034.json
name MMSYN1_0026_1
id_frag BBF10K_000013_1
../data/BBF10K_000013/BBF10K_000013.json
name MMSYN1_0804_1
id_frag BBF10K_000355_1
../data/BBF10K_000355/BBF10K_000355.json
name MMSYN1_0804_2
id_frag BBF10K_000355_2
../data/BBF10K_000355/BBF10K_000355.json
name MMSYN1_0804_3
id_frag BBF10K_000355_3
../data/BBF10K_000355/BBF10K_000355.json
name MMSYN1_0813_1
id_frag BBF10K_000361_1
../data/BBF10K_000361/BBF10K_0003

name MMSYN1_0095_1
id_frag BBF10K_000039_1
../data/BBF10K_000039/BBF10K_000039.json
name MMSYN1_0095_2
id_frag BBF10K_000039_2
../data/BBF10K_000039/BBF10K_000039.json
name MMSYN1_0230_1
id_frag BBF10K_000098_1
../data/BBF10K_000098/BBF10K_000098.json
name MMSYN1_0443_1
id_frag BBF10K_000220_1
../data/BBF10K_000220/BBF10K_000220.json
name MMSYN1_0003_1
id_frag BBF10K_000003_1
../data/BBF10K_000003/BBF10K_000003.json
name MMSYN1_0652_1
id_frag BBF10K_000294_1
../data/BBF10K_000294/BBF10K_000294.json
name MMSYN1_0657_1
id_frag BBF10K_000299_1
../data/BBF10K_000299/BBF10K_000299.json
name MMSYN1_0706_1
id_frag BBF10K_000323_1
../data/BBF10K_000323/BBF10K_000323.json
name MMSYN1_0262_1
id_frag BBF10K_000113_1
../data/BBF10K_000113/BBF10K_000113.json
name MMSYN1_0394_1
id_frag BBF10K_000178_1
../data/BBF10K_000178/BBF10K_000178.json
name MMSYN1_0394_2
id_frag BBF10K_000178_2
../data/BBF10K_000178/BBF10K_000178.json
name MMSYN1_0221_1
id_frag BBF10K_000093_1
../data/BBF10K_000093/BBF10K_0000

name MMSYN1_0644_1
id_frag BBF10K_000288_1
../data/BBF10K_000288/BBF10K_000288.json
name MMSYN1_0282_1
id_frag BBF10K_000119_1
../data/BBF10K_000119/BBF10K_000119.json
name MMSYN1_0688_1
id_frag BBF10K_000315_1
../data/BBF10K_000315/BBF10K_000315.json
name MMSYN1_0046_1
id_frag BBF10K_000025_1
../data/BBF10K_000025/BBF10K_000025.json
name MMSYN1_0412_1
id_frag BBF10K_000193_1
../data/BBF10K_000193/BBF10K_000193.json
name MMSYN1_0412_2
id_frag BBF10K_000193_2
../data/BBF10K_000193/BBF10K_000193.json
name MMSYN1_0412_3
id_frag BBF10K_000193_3
../data/BBF10K_000193/BBF10K_000193.json
name MMSYN1_0308_1
id_frag BBF10K_000137_1
../data/BBF10K_000137/BBF10K_000137.json
name MMSYN1_0429_1
id_frag BBF10K_000207_1
../data/BBF10K_000207/BBF10K_000207.json
name MMSYN1_0347_1
id_frag BBF10K_000151_1
../data/BBF10K_000151/BBF10K_000151.json
name MMSYN1_0415_1
id_frag BBF10K_000196_1
../data/BBF10K_000196/BBF10K_000196.json
name MMSYN1_0415_2
id_frag BBF10K_000196_2
../data/BBF10K_000196/BBF10K_0001

name MMSYN1_0399_3
id_frag BBF10K_000180_3
../data/BBF10K_000180/BBF10K_000180.json
name MMSYN1_0409_1
id_frag BBF10K_000190_1
../data/BBF10K_000190/BBF10K_000190.json
name MMSYN1_0438_1
id_frag BBF10K_000215_1
../data/BBF10K_000215/BBF10K_000215.json
name MMSYN1_0281_1
id_frag BBF10K_000118_1
../data/BBF10K_000118/BBF10K_000118.json
name MMSYN1_0239_1
id_frag BBF10K_000102_1
../data/BBF10K_000102/BBF10K_000102.json
name MMSYN1_0239_2
id_frag BBF10K_000102_2
../data/BBF10K_000102/BBF10K_000102.json
name MMSYN1_0447_1
id_frag BBF10K_000223_1
../data/BBF10K_000223/BBF10K_000223.json
name MMSYN1_0410_1
id_frag BBF10K_000191_1
../data/BBF10K_000191/BBF10K_000191.json
name MMSYN1_0166_1
id_frag BBF10K_000075_1
../data/BBF10K_000075/BBF10K_000075.json
name MMSYN1_0531_1
id_frag BBF10K_000253_1
../data/BBF10K_000253/BBF10K_000253.json
name MMSYN1_0531_2
id_frag BBF10K_000253_2
../data/BBF10K_000253/BBF10K_000253.json
name MMSYN1_0531_3
id_frag BBF10K_000253_3
../data/BBF10K_000253/BBF10K_0002

name MMSYN1_0420_1
id_frag BBF10K_000200_1
../data/BBF10K_000200/BBF10K_000200.json
name MMSYN1_0444_1
id_frag BBF10K_000221_1
../data/BBF10K_000221/BBF10K_000221.json
name MMSYN1_0444_2
id_frag BBF10K_000221_2
../data/BBF10K_000221/BBF10K_000221.json
name MMSYN1_0081_1
id_frag BBF10K_000037_1
../data/BBF10K_000037/BBF10K_000037.json
name MMSYN1_0140_1
id_frag BBF10K_000059_1
../data/BBF10K_000059/BBF10K_000059.json
name MMSYN1_0146_1
id_frag BBF10K_000065_1
../data/BBF10K_000065/BBF10K_000065.json
name MMSYN1_0222_1
id_frag BBF10K_000094_1
../data/BBF10K_000094/BBF10K_000094.json
name MMSYN1_0222_2
id_frag BBF10K_000094_2
../data/BBF10K_000094/BBF10K_000094.json
name MMSYN1_0439_1
id_frag BBF10K_000216_1
../data/BBF10K_000216/BBF10K_000216.json
name MMSYN1_0439_2
id_frag BBF10K_000216_2
../data/BBF10K_000216/BBF10K_000216.json
name MMSYN1_0441_1
id_frag BBF10K_000218_1
../data/BBF10K_000218/BBF10K_000218.json
name MMSYN1_0651_1
id_frag BBF10K_000293_1
../data/BBF10K_000293/BBF10K_0002

In [309]:
import datetime
datetime.date(2010, 6, 16).isocalendar()[1]


24

In [310]:
name = "MMSYN1_0797_link_MMSYN1_0317_1"
gene = name[:-2]
print(gene)
fragment = name[-1]
genes = []

if "link" in gene:
    genes = gene.split("_link_")
    print(genes)
    for gene in genes:
        




MMSYN1_0797_link_MMSYN1_0317
['MMSYN1_0797', 'MMSYN1_0317']


In [315]:
import md2py

file = "../raw_files/documentation/submission.md"

f = open(file, 'r')
print(f)
toc = md2py(file)

<_io.TextIOWrapper name='../raw_files/documentation/submission.md' mode='r' encoding='UTF-8'>


TypeError: 'module' object is not callable